# **Grab Hydromet Data**

By: Bridget Bittmann

Date created: 05/24/2022

Date modified: 08/11/2022

Purpose: This script uses a pre-existing csv of hydromet data and takes summary statistics on Lucky Peak inflows and maximum reservoir storage for the three reservoirs in the system based on the water year. 


In [1]:
## --------------- ## 
## IMPORT PACKAGES ## 
## --------------- ## 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Navigate to your directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Navigate to workspace

%cd /content/drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrigation_companies/  lcmap_files/   subset_test_shp/
diversion_timeseries/  irrig_lbrb/            masked/
extra_figures/         IrrMapper/             output_files/
hydromet_data/         LBRB_shp/              POUs/


In [12]:
## --------------------- ## 
## IMPORT HDYROMET FILES ##
## --------------------- ## 

hydromet = pd.read_csv('hydromet_data/reservoir_data.csv', parse_dates=['DateTime'])
hydromet = hydromet[hydromet['DateTime']>='01-01-1986'].drop('Unnamed: 0', axis=1)

In [16]:
hydromet

,DateTime,luc_id,luc_qu,luc_fb,luc_qt,luc_xqd,luc_qsd,luc_af,luc_qd,and_id,...,ark_pu,ark_px,ark_pp,ark_mm,ark_mx,ark_mn,mo,year,total_storage,wy
25202,1986-10-01,NaN,1801.43,2942.02,NaN,NaN,NaN,38406.11,2000.00,NaN,...,NaN,0.0,0.02,45.56,56.33,38.01,10,1986,531122.11,1987
25203,1986-10-02,-19997.80,1418.35,2942.11,NaN,NaN,NaN,38520.00,1970.00,NaN,...,NaN,0.0,0.01,49.88,62.07,38.01,10,1986,530027.94,1987
25204,1986-10-03,2337.99,1382.81,2943.64,NaN,NaN,NaN,39210.00,1950.00,NaN,...,NaN,0.0,0.00,49.83,60.97,37.94,10,1986,528902.93,1987
25205,1986-10-04,2306.82,1319.85,2943.12,NaN,NaN,NaN,39830.00,1950.00,NaN,...,NaN,NaN,0.00,52.75,65.61,41.40,10,1986,527653.04,1987
25206,1986-10-05,2286.37,1324.05,2943.56,NaN,NaN,NaN,40400.00,1950.00,NaN,...,NaN,0.0,0.01,54.53,66.40,43.28,10,1986,526411.45,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37616,2020-09-26,138.62,767.82,2976.01,2761.31,2767.78,33.93,89398.02,2761.31,NaN,...,14.49,NaN,0.08,57.59,65.40,51.30,9,2020,410426.02,2020
37617,2020-09-27,166.17,787.30,2972.98,2747.53,2755.30,26.63,84277.97,2747.53,NaN,...,14.49,NaN,0.00,54.21,68.30,41.00,9,2020,406537.97,2020
37618,2020-09-28,444.13,709.32,2970.22,2745.66,2751.78,27.54,79712.95,2745.66,NaN,...,14.49,NaN,0.00,57.04,72.60,42.50,9,2020,402498.95,2020
37619,2020-09-29,423.58,737.17,2967.40,2727.68,2733.77,30.11,75142.84,2727.68,NaN,...,14.49,NaN,0.00,59.89,81.70,42.60,9,2020,398550.84,2020


In [57]:
## ---------------------------------------------- ## 
## SUM INFLOWS AND RESERVOIR FILL FOR WATER YEARS ## 
## ---------------------------------------------- ## 

years = np.arange(1987, 2021)

hydromet = hydromet[hydromet['wy'].isin(years)]

LP_inflows = pd.DataFrame(hydromet.groupby('wy').luc_id.sum())
max_fill = hydromet.groupby('wy').total_storage.max()

carryover = []
for i in years:
  subset = hydromet[hydromet['wy'] == i]
  date = str(i-1)+'-10-31'
  carry = int(subset['total_storage'][subset['DateTime'] == date].values)
  carryover.append([carry, i])

carryover = pd.DataFrame(carryover, columns=['Carryover','Year'])

hydromet_new = LP_inflows.merge(max_fill, on='wy', how='left').reset_index()
hydromet_new = hydromet_new.rename({'luc_id':'LP_inflows', 
                                    'total_storage':'Max_Fill',
                                    'wy':'Year'}, axis=1)
hydromet_new = hydromet_new.merge(carryover, on='Year', how='left')

hydromet_new.to_csv('hydromet_data/mode_input_hydromet.csv')